# AutoGluon: Time Series Forecasting with Chronos

## Objective
This notebook demonstrates **time series forecasting** using AutoGluon with Chronos, a foundation model for time series. Chronos leverages pre-trained models for improved forecasting.

## Use Case
Chronos-based forecasting is useful for:
- Zero-shot forecasting on new time series
- Limited historical data scenarios
- Multiple diverse time series patterns
- Transfer learning for time series
- Rapid prototyping without extensive tuning

## Key Features
- Pre-trained on large-scale time series data
- Works well with limited data
- Generalizes across different domains
- Handles various time series patterns
- Reduced training time compared to from-scratch models

In [ ]:
# Install AutoGluon
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil

In [ ]:
# Load dataset
# TODO: Upload your time series dataset or use URL
# Time series data should have:
# - Timestamp column (date/datetime)
# - Target value column (what you want to forecast)
# - Optional: Item/entity ID for multiple time series

# Chronos works particularly well with:
# - Short time series (limited historical data)
# - Multiple heterogeneous time series
# - Time series from new domains

# Example: train_data = TabularDataset('path/to/timeseries_data.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print("\nColumn types:")
    print(train_data.dtypes)
    print("\nSample data:")
    print(train_data.head())

In [ ]:
# Set label column (target variable to forecast)
LABEL = 'target'  # TODO: Replace with your target column name

In [ ]:
# Auto-detect problem type based on label
# Time series forecasting is typically a regression problem
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default to regression for time series
    problem_type = 'regression'
    eval_metric = 'rmse'

print(f"Problem Type: {problem_type}")
print(f"Evaluation Metric: {eval_metric}")
print("\nNote: Using Chronos foundation model for time series forecasting.")

In [ ]:
# Train the model with Chronos
# Chronos is a pre-trained foundation model for time series
# It can be used directly or fine-tuned on your data

predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-chronos-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900,
    # For Chronos-based forecasting:
    # - Can use pre-trained Chronos models
    # - Optional fine-tuning on your data
    # - Works well even with limited training data
)

print("Model training completed!")
print("Chronos-based model leverages pre-trained time series knowledge.")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions (Forecasted Values):")
    print(predictions.head())
    
    # Compare with actual values if available
    if LABEL in test_data.columns:
        results = pd.DataFrame({
            'Actual': test_data[LABEL],
            'Predicted': predictions
        })
        print("\nActual vs Predicted:")
        print(results.head(10))
        
    # Chronos advantages
    print("\nChronos Model Advantages:")
    print("- Pre-trained on diverse time series data")
    print("- Works well with limited historical data")
    print("- Generalizes across different domains")
    print("- Reduced training time")
    print("- Good zero-shot performance")

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-chronos-model'
zip_filename = 'autogluon_chronos_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")